In [ ]:
def transformArray():
    trgPos = lev, row, col, 1
    srcPos = myproduct(invMat, trgPos)
    trgArr(lev, row, col) = interpolNN(srcArr, srcPos)

def main():
    # Something seomthing load data